In [1]:
install.packages("dplyr")
install.packages("caret")
install.packages("caTools")
install.packages("parsnip")
install.packages("corrplot")

Installing package into ‘/home/avind/R/x86_64-pc-linux-gnu-library/4.3’
(as ‘lib’ is unspecified)



Installing package into ‘/home/avind/R/x86_64-pc-linux-gnu-library/4.3’
(as ‘lib’ is unspecified)



In [1]:
library(dplyr)
library(caret)
library(caTools)
library(parsnip)
library(broom)
library(corrplot)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: ggplot2

Loading required package: lattice

corrplot 0.92 loaded



In [2]:
dataset <- read.csv("./dataset.csv", header = TRUE)

In [3]:
# Basic summarizations
dim(dataset)
head(dataset)

[1] 1470   35

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,⋯,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,⋯,1,80,0,8,0,1,6,4,0,5
2,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,⋯,4,80,1,10,3,3,10,7,1,7
3,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,⋯,2,80,0,7,3,3,0,0,0,0
4,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,⋯,3,80,0,8,3,3,8,7,3,0
5,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,⋯,4,80,1,6,3,3,2,2,2,2
6,32,No,Travel_Frequently,1005,Research & Development,2,2,Life Sciences,1,8,⋯,3,80,0,8,2,2,7,7,3,6


In [4]:
# Removing obvious useless columns
dataset$EmployeeNumber <- NULL
dataset$StandardHours <- NULL
dataset$Over18 <- NULL
dataset$EmployeeCount <- NULL

In [5]:
# Splitting the dataset into the Training set and Test set
model_dataset <- dataset
model_dataset$Attrition <- as.factor(model_dataset$Attrition)

set.seed(100)
split_ratio <- sample.split(model_dataset, SplitRatio = 0.7)

training_set <-  subset(model_dataset, split_ratio=="TRUE")
testing_set <- subset(model_dataset, split_ratio=="FALSE")

In [6]:
lr_model <- logistic_reg() %>%
    set_engine("glm") %>%
    set_mode("classification") %>%
    fit(Attrition ~ ., data = training_set)

tidy(lr_model)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-1.014717e+01,2.191850e+03,-0.004629499,9.963062e-01
Age,-3.949925e-02,1.755642e-02,-2.249846603,2.445868e-02
BusinessTravelTravel_Frequently,1.683936e+00,5.309922e-01,3.171300462,1.517581e-03
BusinessTravelTravel_Rarely,8.109927e-01,4.867602e-01,1.666103290,9.569284e-02
DailyRate,-1.657446e-04,2.874360e-04,-0.576631404,5.641885e-01
DepartmentResearch & Development,1.525090e+01,2.191850e+03,0.006958004,9.944484e-01
DepartmentSales,1.641323e+01,2.191850e+03,0.007488303,9.940253e-01
DistanceFromHome,5.584796e-02,1.416061e-02,3.943894881,8.016881e-05
Education,5.755325e-02,1.116497e-01,0.515480384,6.062174e-01


In [41]:
prediction_results <- predict(lr_model, new_data=testing_set, type = "class")

# Confusion Matrix
result_matrix <- table(testing_set$Attrition, prediction_results$.pred_class)
confusionMatrix(result_matrix)

Confusion Matrix and Statistics

     
       No Yes
  No  350  44
  Yes  42  37
                                          
               Accuracy : 0.8182          
                 95% CI : (0.7804, 0.8519)
    No Information Rate : 0.8288          
    P-Value [Acc > NIR] : 0.7513          
                                          
                  Kappa : 0.3531          
                                          
 Mcnemar's Test P-Value : 0.9141          
                                          
            Sensitivity : 0.8929          
            Specificity : 0.4568          
         Pos Pred Value : 0.8883          
         Neg Pred Value : 0.4684          
             Prevalence : 0.8288          
         Detection Rate : 0.7400          
   Detection Prevalence : 0.8330          
      Balanced Accuracy : 0.6748          
                                          
       'Positive' Class : No              
                                          